In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import nltk
from nltk.corpus import stopwords

C:\Users\raghu\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raghu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
# Load Data
df = pd.read_csv("C:\\Users\\raghu\\Downloads\\Tweets (1).csv")  # Replace with your file path

In [4]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
df = df[['tweet_id', 'airline_sentiment', 'text']]

In [7]:
# Preprocess Text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stop words
    return text

df['text'] = df['text'].apply(preprocess_text)


In [10]:
# Encode Labels
label_encoder = LabelEncoder()
df['airline_sentiment'] = label_encoder.fit_transform(df['airline_sentiment'])

# Tokenization and Padding
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=100)
y = tf.keras.utils.to_categorical(df['airline_sentiment'], num_classes=3)

# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Model Development
model = Sequential([
    Embedding(input_dim=20000, output_dim=128, input_length=100),
    SpatialDropout1D(0.2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

C:\Users\raghu\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1)

Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 31s 141ms/step - accuracy: 0.6441 - loss: 0.8522 - val_accuracy: 0.7777 - val_loss: 0.5534
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 25s 139ms/step - accuracy: 0.8154 - loss: 0.4774 - val_accuracy: 0.8016 - val_loss: 0.5143
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 26s 140ms/step - accuracy: 0.8855 - loss: 0.3190 - val_accuracy: 0.7944 - val_loss: 0.5397
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 27s 146ms/step - accuracy: 0.9177 - loss: 0.2374 - val_accuracy: 0.7971 - val_loss: 0.5992
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 27s 145ms/step - accuracy: 0.9365 - loss: 0.1823 - val_accuracy: 0.7927 - val_loss: 0.6533
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 27s 145ms/step - accuracy: 0.9481 - loss: 0.1529 - val_accuracy: 0.7876 - val_loss: 0.7379
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 26s 140ms/step - accuracy: 0.9577 - loss: 0.1224 - val_accuracy: 0.7725 - val_loss: 0.8150
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 26s 140ms/step - accuracy: 0.9625 - loss: 0

In [14]:
# Evaluate Model
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Accuracy: {accuracy:.2f}')

Validation Accuracy: 0.76
